In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import cm
import fastf1 as ff1
from fastf1.core import Laps
from fastf1 import utils
from fastf1 import plotting
plotting.setup_mpl()
from timple.timedelta import strftimedelta
import datetime
from datetime import datetime

In [2]:
import os

cache_dir = 'Desktop/Thesis/Documents/GitHub/ThesisF1/Cache'

if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

ff1.Cache.enable_cache(cache_dir)

In [3]:
schedule2019 = ff1.get_event_schedule(2019)
schedule2020 = ff1.get_event_schedule(2020)
schedule2021 = ff1.get_event_schedule(2021)
schedule2022 = ff1.get_event_schedule(2022)

In [4]:
all_data = pd.DataFrame()

## Race 1

In [4]:
# Load race data
race_2019 = ff1.get_session(2019, 1, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = "Australia"
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[0])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 1, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 1)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "1"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'1': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "1"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/soniamuthi/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/soniamuthi/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], so

NameError: name 'all_data' is not defined

## Race 2

In [7]:
# Load race data
race_2019 = ff1.get_session(2019, 2, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][1]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[1])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 2, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 2)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "2"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "2"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '16', '33', '5', '4', '7', '10', '23', '11', '99', '26', '20', '18', '63', '88', '27', '3', '55', '8']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 3

In [8]:
# Load race data
race_2019 = ff1.get_session(2019, 3, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

data = data.drop(data.index[[9]])

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][2]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[2])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 3, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps

aux = aux[aux['DriverNumber'] != "23"]
data = data[data["DriverNumber"] != "23"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 3)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "3"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'3': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "3"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Chinese Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '5', '33', '16', '10', '3', '11', '7', '23', '8', '18', '20', '55', '99', '63', '88', '4', '26', '27']
core           INFO 	Loading data for Chinese Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 4

In [9]:
# Load race data
race_2019 = ff1.get_session(2019, 4, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][3]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[3])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 4, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 4)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "4"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'4': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "4"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '44', '5', '33', '16', '11', '55', '4', '18', '7', '23', '99', '20', '27', '63', '88', '10', '8', '26', '3']
core           INFO 	Loading data for Azerbaijan Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for

## Race 5

In [10]:
# Load race data
race_2019 = ff1.get_session(2019, 5, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][4]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[4])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 5, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 5)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "5"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'5': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "5"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Spanish Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '33', '5', '16', '10', '20', '55', '26', '8', '23', '3', '27', '7', '11', '99', '63', '88', '18', '4']
core           INFO 	Loading data for Spanish Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 6

In [11]:
# Load race data
race_2019 = ff1.get_session(2019, 6, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][5]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[5])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 6, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 6)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "6"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'6': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "6"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Monaco Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '5', '77', '33', '10', '55', '26', '23', '3', '8', '4', '11', '27', '20', '63', '18', '7', '88', '99', '16']
core           INFO 	Loading data for Monaco Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_

## Race 7

In [12]:
# Load race data
race_2019 = ff1.get_session(2019, 7, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][6]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[6])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 7, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 7)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "7"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'7': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "7"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Canadian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '5', '16', '77', '33', '3', '27', '10', '18', '26', '55', '11', '99', '8', '7', '63', '20', '88', '23', '4']
core           INFO 	Loading data for Canadian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for tim

## Race 8

In [13]:
# Load race data
race_2019 = ff1.get_session(2019, 8, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][7]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[7])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 8, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 8)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "8"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'8': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "8"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for French Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '16', '33', '5', '55', '7', '27', '4', '10', '3', '11', '18', '26', '23', '99', '20', '88', '63', '8']
core           INFO 	Loading data for French Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_

## Race 9

In [14]:
# Load race data
race_2019 = ff1.get_session(2019, 9, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][8]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[8])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 9, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 9)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "9"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'9': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "9"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Austrian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '16', '77', '5', '44', '4', '10', '55', '7', '99', '11', '3', '27', '18', '23', '8', '26', '63', '20', '88']
core           INFO 	Loading data for Austrian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for tim

## Race 10

In [15]:
# Load race data
race_2019 = ff1.get_session(2019, 10, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][9]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[9])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 10, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 10)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "10"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'10': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "10"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for British Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '16', '10', '33', '55', '3', '7', '26', '27', '4', '23', '18', '63', '88', '5', '11', '99', '8', '20']
core           INFO 	Loading data for British Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 11

In [16]:
# Load race data
race_2019 = ff1.get_session(2019, 11, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][10]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[10])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 11, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps

aux = pd.DataFrame(aux)
aux = aux[aux['DriverNumber'] != "11"]
data = data[data["DriverNumber"] != "11"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))


# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 11)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "11"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'11': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables

data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "11"
all_data = pd.concat([all_data, data])



core           INFO 	Loading data for German Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '5', '26', '18', '55', '23', '8', '20', '44', '88', '63', '7', '99', '10', '77', '27', '16', '4', '3', '11']
core           INFO 	Loading data for German Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_

## Race 12

In [17]:
# Load race data
race_2019 = ff1.get_session(2019, 12, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][11]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[11])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 12, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 12)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "12"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'12': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "12"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Hungarian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '5', '16', '55', '10', '7', '77', '4', '23', '11', '27', '20', '3', '26', '63', '18', '99', '88', '8']
core           INFO 	Loading data for Hungarian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for t

## Race 13

In [18]:
# Load race data
race_2019 = ff1.get_session(2019, 13, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][12]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[12])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 13, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps

aux = aux[aux['DriverNumber'] != "33"]
data = data[data["DriverNumber"] != "33"] 
aux = aux[aux['DriverNumber'] != "55"]
data = data[data["DriverNumber"] != "55"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})

# Complete missing data
data.loc[data['DriverNumber'] == '88', 'BestQualiTime'] = pd.Timedelta('0 days 00:01:50.838')


    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 13)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "13"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'13': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "13"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Belgian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '44', '77', '5', '23', '11', '26', '27', '10', '18', '4', '20', '8', '3', '63', '7', '88', '99', '55', '33']
core           INFO 	Loading data for Belgian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 14

In [19]:
# Load race data
race_2019 = ff1.get_session(2019, 14, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][13]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[13])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 14, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 14)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "14"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'14': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "14"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Italian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '77', '44', '3', '27', '23', '11', '33', '99', '4', '10', '18', '5', '63', '7', '8', '88', '20', '26', '55']
core           INFO 	Loading data for Italian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 15

In [20]:
# Load race data
race_2019 = ff1.get_session(2019, 15, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][14]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[14])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 15, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 15)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "15"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'15': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "15"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Singapore Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '16', '33', '44', '77', '23', '4', '10', '27', '99', '8', '55', '18', '3', '26', '88', '20', '7', '11', '63']
core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for t

## Race 16

In [21]:
# Load race data
race_2019 = ff1.get_session(2019, 16, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][15]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[15])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 16, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps
aux = aux[aux['DriverNumber'] != "26"]
data = data[data["DriverNumber"] != "26"] 
aux = aux[aux['DriverNumber'] != "8"]
data = data[data["DriverNumber"] != "8"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})

import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 16)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "16"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'16': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "16"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Russian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '16', '33', '23', '55', '11', '4', '20', '27', '18', '26', '7', '10', '99', '88', '63', '5', '3', '8']
core           INFO 	Loading data for Russian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 17

In [22]:
# Load race data
race_2019 = ff1.get_session(2019, 17, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][16]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[16])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 17, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 17)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "17"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'17': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "17"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Japanese Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '5', '44', '23', '55', '16', '10', '11', '18', '26', '4', '7', '8', '99', '20', '63', '88', '33', '3', '27']
core           INFO 	Loading data for Japanese Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for tim

## Race 18

In [23]:
# Load race data
race_2019 = ff1.get_session(2019, 18, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][17]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[17])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 18, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 18)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "18"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'18': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "18"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Mexican Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '5', '77', '16', '23', '33', '11', '3', '10', '27', '26', '18', '55', '99', '20', '63', '8', '88', '7', '4']
core           INFO 	Loading data for Mexican Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 19

In [24]:
# Load race data
race_2019 = ff1.get_session(2019, 19, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][18]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[18])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 19, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 19)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "19"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'19': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "19"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for United States Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '44', '33', '16', '23', '3', '4', '55', '27', '11', '7', '26', '18', '99', '8', '10', '63', '20', '88', '5']
core           INFO 	Loading data for United States Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached da

## Race 20

In [25]:
# Load race data
race_2019 = ff1.get_session(2019, 20, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][19]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[19])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 20, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 20)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "20"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'20': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "20"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Brazilian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '10', '55', '7', '99', '3', '44', '4', '11', '26', '20', '63', '8', '23', '27', '88', '5', '16', '18', '77']
core           INFO 	Loading data for Brazilian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for t

## Race 21

In [26]:
# Load race data
race_2019 = ff1.get_session(2019, 21, 'R')
race_2019.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2019.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)


# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2019.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2019["Country"][20]
data["Year"] = 2019
dati = schedule2019["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[20])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2019 = ff1.get_session(2019, 21, 'Q')
qualification1_2019.load()

# Add the best qualifications lap
fastest_laps = qualification1_2019.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2019.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2019.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2019) & (races["round"] == 21)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2019.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
wins = pd.read_csv("podiums2019.csv", sep = ";")
wins = wins[["Driver", "21"]]

data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

data = data.drop(["Driver"], axis = 1)

data = data.rename(columns={'21': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2019.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2019.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "21"
all_data = pd.concat([all_data, data])

core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '16', '77', '5', '23', '11', '4', '26', '55', '3', '27', '7', '20', '8', '99', '63', '10', '88', '18']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for t

In [27]:
all_data.to_csv(r'2019data.csv', index=True, header=True)

In [28]:
all_data

,Abbreviation,DriverNumber,TeamName,Position,GridPosition,Status,Points,RaceCountry,Year,AgeAtGP,...,Rain,Driver,AverageSpeed,MaxSpeed,AverageRPM,MaxRPM,AverageThrottle,MaxThrottlePct,Brake,raceID
0,GAS,10,Red Bull Racing,11.0,17.0,+1 Lap,0.0,Australia,2019.0,22.0,...,False,10,153.154148,326,7793.417721,12860,49.783810,38.567815,13.413764,1
1,PER,11,Racing Point,13.0,10.0,+1 Lap,0.0,Australia,2019.0,29.0,...,False,11,152.749173,331,7973.695239,13252,50.152656,24.220017,14.542388,1
2,LEC,16,Ferrari,5.0,5.0,Finished,10.0,Australia,2019.0,21.0,...,False,16,155.409580,312,7691.757735,12319,56.245606,3.762081,15.816955,1
3,STR,18,Racing Point,9.0,16.0,+1 Lap,2.0,Australia,2019.0,20.0,...,False,18,153.104527,323,7737.558799,13073,49.155575,3.580463,13.238633,1
4,MAG,20,Haas F1 Team,6.0,7.0,Finished,8.0,Australia,2019.0,26.0,...,False,20,155.144905,314,7912.993287,12771,58.128754,7.825777,20.419018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,RAI,7,Alfa Romeo Racing,13.0,17.0,+1 Lap,0.0,UAE,2019.0,40.0,...,False,7,139.389846,344,7402.660745,12840,54.256140,8.407731,25.484715,21
16,BOT,77,Mercedes,4.0,20.0,Finished,12.0,UAE,2019.0,30.0,...,False,77,141.719842,339,7583.554604,13233,47.303334,0.937866,16.962756,21
17,GRO,8,Haas F1 Team,15.0,15.0,+1 Lap,0.0,UAE,2019.0,33.0,...,False,8,139.600355,341,7576.323053,12822,43.428653,0.399796,16.508853,21
18,KUB,88,Williams,19.0,19.0,+2 Laps,0.0,UAE,2019.0,35.0,...,False,88,136.879160,327,7442.653200,12633,47.375056,1.908799,17.461749,21


## Complete missing values

In [29]:
# no pit stops
all_data['AvgPitTime'] = all_data['AvgPitTime'].fillna(pd.Timedelta("0 days"))

all_data['PitstopNo'] = all_data['PitstopNo'].fillna(0)

In [30]:
# Missing qualification times
# VET Germany
all_data.loc[(all_data['Abbreviation'] == 'VET') & (all_data['RaceCountry'] == 'Germany'), 'BestQualiTime'] = pd.Timedelta('0 days 00:01:17.285')

# MAG Japan
all_data.loc[(all_data['Abbreviation'] == 'MAG') & (all_data['RaceCountry'] == 'Japan'), 'BestQualiTime'] = pd.Timedelta('0 days 00:01:34.593')

# KUB Japan
all_data.loc[(all_data['Abbreviation'] == 'KUB') & (all_data['RaceCountry'] == 'Japan'), 'BestQualiTime'] = pd.Timedelta('0 days 00:01:34.593')


In [31]:
all_data.to_csv(r'2019data.csv', index=True, header=True)